**Absa90.11: Making Benchmark Dataset**

Refs:
https://colab.research.google.com/drive/1WJUl-ciCbqaNLzhItTFbD5caOpGRoTVF#scrollTo=UcOXlh8Eoj7F

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import getpass, os
username = getpass.getpass("Github username: ")
password = getpass.getpass("Github password: ")
os.environ["CRED"] = f"{username}:{password}"
!git clone https://${CRED}@github.com/lopentu/casa
del username, password
!pip install -r /content/casa/requirements.txt

Github username: ··········
Github password: ··········
Cloning into 'casa'...
remote: Enumerating objects: 772, done.
remote: Counting objects: 100% (772/772), done.
remote: Compressing objects: 100% (488/488), done.
remote: Total 772 (delta 379), reused 614 (delta 231), pack-reused 0
Receiving objects: 100% (772/772), 4.65 MiB | 16.04 MiB/s, done.
Resolving deltas: 100% (379/379), done.
     |████████████████████████████████| 1.2MB 30.5MB/s 


In [ ]:
%load_ext autoreload
%autoreload 2
import pickle 
import sys
import pandas as pd

In [ ]:
sys.path = ["/content/casa/src"] + sys.path

In [ ]:
workspace = '/content/gdrive/MyDrive/指向情緒案/data'
aspect_tuples_path = f'{workspace}/annot_data/annotated_data_bkup/20210605/aspect_tuples_20210605.csv'
threads_path = f'{workspace}/threads/cht-2020-merged.pkl'

In [ ]:
import casa
with open (threads_path, 'rb') as File:
    threads = pickle.load(File)

In [ ]:
print(type(threads))
print(type(threads[0]))
# print(*threads[:10], sep = '\n')
print(threads[0].main)
print(threads[0].replies)
print(threads[0].get_opinion)
print(threads[0].title)
print(len(threads))

<class 'list'>
<class 'casa.opinion_thread.OpinionThread'>
<Opinion [BabyHome 寶貝家庭親子網] ChenMimi: 華碩超大方 元月起買華碩兩款手機送耳機！...>
[]
<bound method OpinionThread.get_opinion of <OpinionThread[T]: 華碩超大方 元月起買華碩兩款手機送耳機/ChenMimi/0 replie(s)>>
華碩超大方 元月起買華碩兩款手機送耳機
56064


In [ ]:
aspdf = pd.read_csv(aspect_tuples_path)
aspdf

,batch_idx,serial,thread_idx,is_context,ent_norm,attr_norm,evaltext,rating,ent_rawtext,attr_rawtext
0,0,43,3260,True,臺灣之星,[通訊]頻段,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。,1.0,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。,台星的態度就是在等宿主台哥，逸以待勞，準備寄生。
1,0,7,281,False,中華電信,[通訊]國內電信漫遊,很可以,4.0,中華,訊號
2,0,13,932,False,臺灣之星,[通訊]網速,不限速卡上傳可,3.0,台星,21M
3,0,1,114,True,中華電信,NaN,中華、遠傳，蓋一座拆一座,1.0,中華、遠傳，蓋一座拆一座,中華、遠傳，蓋一座拆一座
4,0,10,674,False,臺灣大哥大,[通訊]國內電信漫遊,越來越爛,1.0,台灣大哥大,收訊
...,...,...,...,...,...,...,...,...,...,...
3106,8,5,109,False,臺灣大哥大,[資費]低資費方案,禮券算下來不會輸488,4.0,台哥,499方案
3107,8,5,109,False,遠傳電信,[資費]月租費,NaN,-1.0,遠傳,588
3108,8,3,45,False,中華電信,[資費]續約攜碼,沒有珍惜已經使用的老客戶權益,1.0,中華電信,續約
3109,8,8,194,False,中華電信,[資費]方案活動,夠,3.0,中華,469限速吃到飽


In [ ]:
thread_indices = list(map(int, aspdf['thread_idx'].tolist()))
# get the threads 
ordered_threads = []
for idx in thread_indices:
    ordered_threads.append((idx, threads[idx])) 
assert len(ordered_threads)==len(thread_indices)
# structure the threads
structed_threads = []
Recorded = []
for (idx, th) in ordered_threads:
    thread_dict = {}
    idx = int(idx)
    if idx not in Recorded:
        Recorded.append(idx) 
        thread_dict['idx'] = idx
        thread_dict['title'] = th.title if len(th.title) > 0 else ""
        try:
            thread_dict['main'] = th.main.text 
        except:
            thread_dict['main'] = ""
        
        thread_dict['replies'] = []
        if len(th.replies)> 0:
          for rep in th.replies:
            thread_dict['replies'].append(rep.text)
        structed_threads.append(thread_dict)

In [ ]:
import random
# print(aspdf.nunique())
print('How many threads have aspect tuples?', len(structed_threads))
r = random.randint(0, len(structed_threads))
print(structed_threads[r]['idx'])
print(structed_threads[r]['title'])
print(structed_threads[r]['main'])
print(structed_threads[r]['replies'])

How many threads have aspect tuples? 1463
10595
[新品介紹]Samsung S20台星預購未來可以免費升級5G

['要買5G手機體驗5G，找中華會比較有信心 學生588或大4G1399都可以體驗，買旗艦配中華可用頻段也較多']



Structure: 
```
[{
  thread: {
    idx: <int>,  
    title: "<title str>", 
    main: "<main text>", 
    reply: ["<reply text>", ...]},
  aspects: [
    {
       entity: "<ent_norm>", 
       attribute: "<attr_norm>",
       evaltext: "<evaltext>",
       rating: <int>,
       context_flag: <bool>
    }
  ]
},...]
```



In [ ]:
df = aspdf.fillna(-1) # -1 代表原格子為NaN
entites = df['ent_norm'].tolist()
attributes = df['attr_norm'].tolist()
evaltexts = df['evaltext'].tolist()
ratings = list(map(int, df['rating'].tolist())) 
context_flags = list(map(bool, df['is_context'].tolist()))
assert len(entites) == len(attributes) == len(evaltexts) == len(ratings) == len(context_flags)
print('How many aspect tuples in total?', len(entites))
structed_aspects = {}
count = 0
for (idx, ent, attr, evt, rating, ctx_flag) in zip(thread_indices, entites, attributes, evaltexts, ratings, context_flags):
    if idx not in structed_aspects:
        structed_aspects[idx] = []
    if evt != -1 or rating != -1:
      aspect_tuple_dict = {}
      aspect_tuple_dict['entity'] = ent
      aspect_tuple_dict['attribute'] = attr
      aspect_tuple_dict['evaltext'] = evt
      aspect_tuple_dict['rating'] = rating
      aspect_tuple_dict['context_flag'] = ctx_flag 
      structed_aspects[idx].append(aspect_tuple_dict)
      count+=1
print('After cleaning, how many aspect tuples in total?', count)

How many aspect tuples in total? 3111
After cleaning, how many aspect tuples in total? 2995


In [ ]:
print(structed_aspects[15603])

[{'entity': '臺灣大哥大', 'attribute': '[通訊]網速', 'evaltext': '超爛', 'rating': 1, 'context_flag': False}, {'entity': '臺灣大哥大', 'attribute': '[通訊]網速', 'evaltext': '爛', 'rating': 1, 'context_flag': False}, {'entity': '臺灣大哥大', 'attribute': '[通訊]涵蓋', 'evaltext': '真的差', 'rating': -1, 'context_flag': False}]


In [ ]:
length_dict = {key: len(value) for key, value in structed_aspects.items()}
for key, value in length_dict.items():
    print(key, value)

In [ ]:
Datasets = []
for str_th in structed_threads:
    merge_dict = {}
    thread_idx = str_th['idx']
    # get the corresponding list of aspects 
    corres_aspects = structed_aspects[thread_idx]
    merge_dict['thread'] = str_th
    merge_dict['aspects'] = corres_aspects
    Datasets.append(merge_dict)
print(len(Datasets)) 

1463


 ### **Check dataset content**

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# Writes the list of dictionaries into json 
import json
import pickle
# workspace = '/content/gdrive/MyDrive/指向情緒案/data'
with open(f'{workspace}/annot_data/benchmark_set.json', 'w', encoding = 'utf-8') as fout:
    json.dump(Datasets, fout, ensure_ascii=False, indent=2)  
with open(f'{workspace}/annot_data/benchmark_set.pkl', "wb") as fout:
    pickle.dump(Datasets, fout) 
### Testing ### 
# with open(f'{workspace}/annot_data/benchmark_set.pkl', "rb") as fin:
#     Datasets = pickle.load(fin)

In [ ]:
def PrintData(Data):
    print('thread:')
    print('\t-idx:', Data['thread']['idx'])
    print('\t-title:', Data['thread']['title'])
    print('\t-main:', Data['thread']['main'])
    print('\t-replies:\t')
    for rep in Data['thread']['replies']:
      print('\t\t--', rep)

    print('aspects:')
    for i, asp in enumerate(Data['aspects']):
       print(f'\t-aspect {i}:', asp)

In [ ]:
import random
r = random.randint(0, len(Datasets))
PrintData(Datasets[r])

thread:
	-idx: 26202
	-title: [問卦] 低速固網是不是乾脆用4G等5G？
	-main: 
	-replies:	
		-- 對惹，我4月繳1399費率，4G吃到飽用戶。用遠傳1樣，現在用台哥大也1樣，手機換惹3、4隻有了。5G世代8想再花大錢被騙惹
aspects:
	-aspect 0: {'entity': '臺灣大哥大', 'attribute': '[資費]月租費', 'evaltext': '8想再花大錢被騙惹', 'rating': 2, 'context_flag': False}
